# Project 1: Data analysis

## 1a) Bayesian approch

In order to predict the effect of genes on symptoms, we will use a simple Bayesian model. We consider these hypothesis:

$$\mu_0: P(symptoms|genes) = P(symptoms)$$
$$\mu_1: P(symptoms|genes) \neq P(symptoms)$$

In order words, does the genes have any effect on the symptoms or not? If it does, which genomes are the most relevant for the preventing the symptoms? 


In our case our Y is the symptoms and our X is the genes. Y is an 8 bit vector for each observation,with 1 or zero for each bit. X is an 128 with the same charectaristic as Y.


The probability distribution of $ Y $ depends on an unknown parameter which also is stocastic. In this case the unknown parameter is $\theta$ or $ p $. 

The formulation under is an alternative formulation of the the previous hypotesis.Does the parameter that the probability dist of Y depend on, depend on what the value of X is or not. $ \mu_0 $ says that it does not.$\mu_1 $ says it does.
 
$$\mu_0: \theta^0 \sim Beta(\alpha^0 , \beta^0) ,  \ \ \  Y/ \theta^0  $$
$$\mu_1: \theta^{1,x_t} \sim Beta(\alpha^{1,x_t} , \beta^{1,x_t}) , \ \ \ y_t/x_t \sim bernulli(\theta^{1,x_t}) $$
 

We will model the hypothesis in the following way. Given the data, we try to estimate

$$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_1}(D) \cdot \xi(\mu_1)} $$

We can eather set $P(\mu | D)$ to mean $P(\mu_1 | D)$ or  $P(\mu_0 | D)$ depending on which hypothesis we want to check the posterior belief of. 

where $P_{\mu} (D) = P(\mu |D)$, $D = (x_t, y_t,x_{t-1},y_{t-1})_{t=1} ^T $ for a spesific gene, $x_t$ is the value of the gene for observation t ,$x_{t-1}$ is a vector of all the previous  values of the previous observation up to t-1,  $y_t$ is the the value of symptom Y for observation t,$y_{t-1}$ is a vector of all the previous  values of the previous symtoms observation up to t-1 and $\mu$ is one of the hypothesis in the set {$\mu_1$,$\mu_0$}.  We assume that the different observations(rows) of the data are independent of each other. This gives us:
$$ P_{\mu}(D) = P(y_1,...,y_t \cup x_1,...,x_t)$$

$$ = \prod_{i=1}^{t} P(y_i | x_i,x_{i-1} , y_{i-1}) $$


With these assumptions, we can estimate the terms in the $P_{\mu} (D)$ by 
$$ P(y_t | x_t,x_{t-1} , y_{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x_{t-1},y_{t-1})$$

We calculate the likehood of adding a new observation(row) using this formula.It works if we assume that the the next observation is independent from the previous observations.


This is the marginal likelihood(posterior predictive distribution) which is a compound distribution,which is used to calculate $P(y_t | x_{t},x_{t-1}, y_{t-1})$. The probabilty $P(y_t | x_{t})$ is stocatic in $\theta$,so we have to integrate over all values of $\theta$. Every time a new observation is added we calculate $P(y_t | x_{t},x_{t-1}, y_{t-1})$,and recalculate $$ = \prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1}) $$. 

So to calculate $$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $$,we are updating our belief,every time a new observation is added,sequentially. For every update $\xi(\mu)$ is $P(\mu | D)$ from the previous iteration,exept the first iteration where $\xi(\mu)$ cancels out because it's constant.

This integral simplifies to:

$$ P(y_t | x_{t},x_{t-1},y^{t-1}) = \int_{\theta} P_{\theta}(y_t) \cdot \xi(\theta|y^{t-1}) = \frac{\alpha_{t,x_t}}{\alpha_{t,x_t} + \beta_{t,x_t}}$$


Looking at $\mu_1$ we are using the beta-binomial distribution.Its a binomial distribution where  the probability of success at each of n trials is not fixed but comes from a beta distribution. The bernulli distribtuion is a special case of the binomial distribution,where number of trials is equal to 1.

We are doing this because our y's are either 1 or 0.Then it is reasonable to assume that Y/X is bernulli distributed.

It is also conviniant because of its conjugate prior property.Now we can calculate  $P( D/\mu_1) =\prod_{i=1}^{t} P(y_i | x^i ,x^{i-1}, y^{i-1}) $ beacuse we know that the marginal likelihood (posterior predictive dist) is composed of a binomial dist times a beta dist $ P(y_t | x^t , y^{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x^{t-1},y^{t-1})$,which is beta distributed .For each iteration we end up  getting that $P_{\mu} (D)$ is beta distributed.

The same integral calculation is done for $P(D | \mu_0)$ but now the marginal likelihood is independent of the data.

Instead of calculating $P(  D|\mu_1) =\prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1})$ ,we can first calculate $log(P(  D|\mu_1))$ and then do $e^{log(P(  D|\mu_1))}$,same for $P(D | \mu_0)$.

To summarize.We have a set of hypotesis in this case two.We want to find how accurate they are. So we have a prior beleif on the strenght of the hypothses which we express through a probability, and we want to update our belief on the strenght of the hypotesis(posterior belief). In other word what is the probability of the choosen hypotesis given the data.Ony have to compute one of the probabilities because $P(\mu_1 | D)=1-P(\mu_0 | D)$.

The calculation is done sequentially.For each iteration we get a new data observation.A new row of genes.We continously update the posterior belief,for all genes,and put the posterior belief of each gene into a list. The belief get updated through calculating the marginal likelihood every time a new row is added and a product is taken between this marginal likelihood and all the presious marginal likelihood calculations bought for $P(  D|\mu_1)$ and for$P(  D|\mu_0)$ this is then put into the bayes formula $ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $.

We want to find this probability for for each of the genes.This procedure is done for each gene.For each gene if the posterior probabilty is greater then some trechhold we add this gene to the list of important genes that are relvant for explaining that spesific symptom.
 



The decision rule can be defined as

$$ \mu =   \left\{
\begin{array}{ll}
      \mu_1 & P(\mu_1|D) > s \\
      \mu_0 & else
\end{array} 
\right.  $$

where $s$ is a given threshold.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [132]:
obs_data = pd.read_csv("observation_features.csv")
treat_data = pd.read_csv("treatment_features.csv") 
action_data = pd.read_csv("treatment_actions.csv")
outcome_data = pd.read_csv("treatment_outcomes.csv")

In [3]:
symptoms = obs_data.iloc[:,0:10]
age = obs_data.iloc[:,10]
gender = obs_data.iloc[:,11]
income = obs_data.iloc[:,12]
genome = obs_data.iloc[:,13:141]
comorbidities = obs_data.iloc[:,141:147]
vaccination_status = obs_data.iloc[:,147:]

In [4]:
def select_features(X, Y, threshold):
    n_features = X.shape[1]
    n_data =  X.shape[0]
    alpha_b = np.ones([n_features, 2 ])
    beta_b = np.ones([n_features, 2])
    log_p = np.zeros(n_features)
    
    log_null = 0
    alpha = 1
    beta = 1  
    for t in range(n_data):
        p_null = alpha / (alpha + beta)
        log_null += np.log(p_null)*Y[t] + np.log(1-p_null)*(1 - Y[t])
        alpha += Y[t]
        beta += (1 - Y[t])
        for i in range(n_features):
            
                x_ti = int(X[t,i])
                p = alpha_b[i, x_ti] / (alpha_b[i, x_ti] + beta_b[i, x_ti])
                log_p[i] += np.log(p)*Y[t] + np.log(1-p)*(1 - Y[t])
                alpha_b[i, x_ti] += Y[t]
                beta_b[i, x_ti] += (1 - Y[t])
    log_max=np.mean(log_p)
    log_max2=np.mean(log_null)
    log_p=log_p-log_max
    log_null=log_null-log_max2
    p = np.exp(log_p) / (np.exp(log_p) + np.exp(log_null))
    #print((np.exp(log_p) + np.exp(log_null)))

    features = []
    for i in range(n_features):
        if p[i] > threshold:
            features.append(i)
    return features

In [5]:
thresold = 0.8
n_rows,n_symptoms = symptoms.shape
n_rows,n_genomes = genome.shape
symptoms.columns = range(n_symptoms)
genome.columns = range(n_genomes)

In [6]:
X = np.array(genome)
Y = np.array(symptoms)
select_0 = select_features(X,Y[:,1],0.5)

In [131]:
def generate_data():
    X,Y = make_classification(n_samples=99999,n_features=150,n_classes=150,n_informative=150,n_redundant=0)
    return X,Y

X_g , Y_g = generate_data()

## Logistic Regression coefficient with Lasso

Lasso regression is a linear model that uses this cost function:

$$\frac{1}{2N} \sum_{i=1}^{N} (y_{pred}^{(i)} - y_{real}^{(i)} )^2 + \alpha \sum_{j=1}^n |a_j| $$

By trying to minimize the cost function, Lasso regression will automatically select those features that are useful, discarding the useless or redundant features. 
So, the idea of using Lasso regression for feature selection purposes is very simple: we fit a Lasso regression on a scaled version of our dataset and we consider only those features that have a coefficient different from 0. Obviously, we first need to tune α hyperparameter in order to have the right kind of Lasso regression.

In [24]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification

In [122]:
#splitting the data:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y[:,1], test_size=0.33, random_state=42)

In [128]:
sel = SelectFromModel(LogisticRegression(penalty='l1',C=0.1,solver='liblinear'))
sel.fit(X_train,Y_train)

SelectFromModel(estimator=LogisticRegression(C=0.1, penalty='l1',
                                             solver='liblinear'))

In [129]:
X_train_L1 = sel.transform(X_train)
len(X_train_L1[0])

118